# Gallery Example: Hyper/Erl/k Queue

This example demonstrates a HyperExp/Erlang/k queueing system:
- **Arrivals**: Hyperexponential distribution (high variability)
- **Service**: Erlang distribution (low variability)
- **Servers**: k servers (default k=2)
- **Capacity**: Infinite
- **Scheduling**: FCFS

This multiserver queue combines high arrival variability with regular service times.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_hyperlk(k=2): Network {
    """Create Hyper/Erl/k queueing model"""
    val model = Network("Hyper/Erl/k")
// Block 1: nodes
    val source = Source(model, "mySource")
    val queue = Queue(model, "myQueue", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
// Block 2: classes
    val oclass = OpenClass(model, "myClass")
// Balanced HyperExp arrival with mean=1/1.8 and SCV=4
    source.setArrival(oclass, HyperExp.fit_mean_and_scv_balanced(1/1.8, 4))
// Erlang service with mean=1 and SCV=0.25 (order 4)
    queue.setService(oclass, Erlang.fit_mean_and_scv(1, 0.25))
    queue.setNumberOfServers(k)
// Block 3: topology
    model.link(Network.serialRouting(source, queue, sink))
    
    return model
// Create the model with k=2 servers
val model = gallery_hyperlk(k=2)

## About Hyper/Erl/k Queues

This model combines:
- **High arrival variability** (SCV = 4): Bursty traffic
- **Low service variability** (SCV = 0.25): Regular service times
- **Multiple servers**: Parallel processing capacity

Parameters:
- Arrival mean = 1/1.8 ≈ 0.556 → arrival rate λ = 1.8
- Service mean = 1.0 → service rate μ = 1.0 per server
- With k=2 servers: total capacity = 2.0
- Utilization ρ = 1.8 / 2.0 = 0.9

The regular Erlang service helps counteract the high arrival variability.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)